In [9]:
#@title 1. 의존성 패키지 설치
!pip install -q langchain langchain-chroma langchain-openai wikipedia ddgs

# langchain-community 최신 버전 설치 (ddgs 패키지 지원을 위하여)
!pip install -q --force-reinstall git+https://github.com/langchain-ai/langchain-community.git@main#subdirectory=libs/community

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.2 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.2 which is incompatible.
numba 0.60.0 r

In [10]:
!pip show langchain-community

Name: langchain-community
Version: 0.3.27
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [11]:
import os

from langchain.tools import tool
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from google.colab import userdata
import wikipedia

In [12]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [13]:
#@title 2. 도구 1: AI 역사 백과사전 Q&A 도구

# 한국어 위키피디아로 언어 설정
wikipedia.set_lang("ko")

# 지식 베이스로 사용할 문서 주제 목록
topics = ["인공지능", "머신러닝", "딥러닝"]

# 수집된 내용을 저장할 디렉토리 생성
os.makedirs("ai_wiki_docs", exist_ok=True)

for topic in topics:
    print(f"'{topic}' 주제의 위키피디아 문서를 수집합니다...")
    try:
        # 위키피디아에서 해당 주제의 페이지 객체를 가져옴
        page = wikipedia.page(topic, auto_suggest=False)

        # 페이지의 전체 텍스트 내용을 파일로 저장
        with open(f"ai_wiki_docs/{topic}.txt", "w", encoding="utf-8") as f:
            f.write(page.content)
        print(f"'{topic}.txt' 파일 저장 완료.")
    except wikipedia.exceptions.PageError:
        print(f"'{topic}'에 대한 위키피디아 페이지를 찾을 수 없습니다.")
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"'{topic}'에 대한 여러 페이지가 있어 명확한 지정이 필요합니다: {e.options}")

llm = ChatOpenAI(
    model_name="gpt-5-mini",
    api_key=OPENAI_API_KEY,
)
loader = DirectoryLoader(
    "./ai_wiki_docs",
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    persist_directory="./chroma_db_ai_project"
)
retriever = vectorstore.as_retriever()
rephrasing_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "주어진 대화 기록을 바탕으로, 후속 질문을 검색에 사용할 수 있는 독립적인 질문으로 바꾸어주세요."),
])

history_aware_retriever = create_history_aware_retriever(llm, retriever, rephrasing_prompt)
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 AI 전문가입니다. 당신의 임무는 오직 주어진 문맥(context) 정보만을 사용하여 사용자의 질문에 답변하는 것입니다.

- 답변의 근거가 된 출처 문서의 이름을 항상 명시해야 합니다. (예: [출처: 인공지능.txt])

- 문맥 정보에 질문에 대한 답변이 포함되어 있지 않다면, 당신의 기존 지식을 활용하지 말고 반드시 "죄송하지만, 제가 가진 정보 내에서는 답변을 찾을 수 없습니다."라고만 답변하세요.

- 절대 외부 정보를 사용하거나 답변을 지어내지 마세요."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("system", "--- 검색된 문맥 ---\n{context}\n--- 문맥 끝 ---"),
])
document_chain = create_stuff_documents_chain(llm, answer_prompt)
conversational_rag_chain = create_retrieval_chain(history_aware_retriever, document_chain)


@tool
def ai_history_qa_tool(query: str) -> str:
    """'인공지능(AI) 역사', '튜링 테스트', '다트머스 워크숍', '머신러닝', '딥러닝'의 역사나 개념에 대한 질문에 답변할 때 사용합니다.
    이 도구는 신뢰할 수 있는 내부 문서 기반으로 답변합니다."""
    # 대화형 체인이므로, 빈 대화 기록과 함께 호출
    response = conversational_rag_chain.invoke({
        "chat_history": [],
        "input": query
    })
    return response["answer"]

'인공지능' 주제의 위키피디아 문서를 수집합니다...
'인공지능.txt' 파일 저장 완료.
'머신러닝' 주제의 위키피디아 문서를 수집합니다...
'머신러닝.txt' 파일 저장 완료.
'딥러닝' 주제의 위키피디아 문서를 수집합니다...
'딥러닝.txt' 파일 저장 완료.


In [14]:
#@title 3. 도구 2: 실시간 웹 검색 도구
web_search_tool = DuckDuckGoSearchRun()

In [15]:
#@title 4. 도구 정의
tools = [ai_history_qa_tool, web_search_tool]

print("에이전트를 위한 도구 2개 정의 완료:")
print(f"1. {ai_history_qa_tool.name}: {ai_history_qa_tool.description}")
print(f"2. {web_search_tool.name}: {web_search_tool.description}")

에이전트를 위한 도구 2개 정의 완료:
1. ai_history_qa_tool: '인공지능(AI) 역사', '튜링 테스트', '다트머스 워크숍', '머신러닝', '딥러닝'의 역사나 개념에 대한 질문에 답변할 때 사용합니다.
    이 도구는 신뢰할 수 있는 내부 문서 기반으로 답변합니다.
2. duckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.


In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_tools_agent

# 에이전트를 위한 프롬프트 정의
# MessagesPlaceholder는 에이전트의 중간 작업 과정(생각과 관찰)이 저장될 공간이다.
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 주어진 도구를 사용하여 사용자의 질문에 답변하는 유능한 AI 어시턴트입니다."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 에이전트 생성
# OpenAI의 tool-calling 기능에 최적화된 에이전트를 생성한다.
agent = create_openai_tools_agent(llm, tools, prompt)

# 에이전트 실행기(Executor) 생성
# 에이전트가 결정한 행동을 실제로 실행하는 역할을 담당한다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("ReAct 기반 에이전트 생성 완료.")

ReAct 기반 에이전트 생성 완료.


In [17]:
# 테스트 질문 정의
questions = [
    # 1번 질문: 내부 백과사전 도구(ai_history_qa_tool)를 사용해야 하는 경우
    "딥러닝과 머신러닝의 개념의 차이점에 대해 우리 백과사전에 있는 내용으로 설명해주세요.",
    # 2번 질문: 실시간 웹 검색 도구(web_search_tool)를 사용해야 하는 경우
    "OpenAI의 가장 최근 마지막으로 발표된 GPT 모델 이름이 뭔가요?",
    # 3번 질문: 두 도구의 정보가 모두 필요할 수 있는 복합적인 경우
    "다트머스 워크숍의 주요 내용과, 그 행사를 주도했던 존 매카시의 최근 근황에 대해 알려주세요."
]

# 각 질문에 대해 에이전트 실행 및 결과 확인
for i, question in enumerate(questions):
    print(f"\n{'='*20} 질문 {i+1} 실행 {'='*20}")
    print(f"사용자 질문: {question}")
    print("-" * 50)

    # 에이전트 실행기를 호출
    response = agent_executor.invoke({"input": question})

    print("-" * 50)
    print(f"최종 답변: {response['output']}")
    print(f"{'='*55}")


==================== 질문 1 실행 ====================
사용자 질문: 딥러닝과 머신러닝의 개념의 차이점에 대해 우리 백과사전에 있는 내용으로 설명해주세요.
--------------------------------------------------


> Entering new AgentExecutor chain...

Invoking: `ai_history_qa_tool` with `{'query': '딥러닝과 머신러닝의 개념의 차이점에 대해 백과사전 스타일로 설명해 주세요'}`


죄송하지만, 제가 가진 정보 내에서는 답변을 찾을 수 없습니다.
Invoking: `duckduckgo_search` with `{'query': 'difference between deep learning and machine learning encyclopedia Korean description'}`


by CD Yi · 2021 · Cited by 5 — This paper analyzes the feasibility of using machine learning and deep learning methods to forecast the income and employment using the strategic industries. by ML Ali · 2025 · Cited by 11 — Our results indicate that deep learning models, particularly CNN and LSTM , outperform traditional machine learning approaches in cyber threat detection, ... Jul 3, 2025 — ... "Artificial intelligence" was defined as a machine learning (ML) or deep learning (DL) tool that identifies patterns resulting in a pre